In [1]:
import numpy as np
from numpy.random import choice
import pandas as pd
from generate_prob_chart import generate_prob_chart

Let $X_{i, j}$ be the count of trying StarForce at $j$ while enhancing StarForce $i \rightarrow i+1$.
+ When $i=j$, $X_{i, j} = X_{i, i} = \frac{1}{p_{i, s}}$.
+ When $j < i$,
\begin{align*}
    X_{i, j} &= p_{i, r}X_{i, j} + p_{i, f}[ I(j=i-1) + p_{i-1, s}X_{i, j} + p_{i-1, r}(X_{i-1, j}+X_{i, j}) \\
    &\quad+ p_{i-1, f}(I(j=i-2)+X_{i-1, j}+X_{i, j}) + p_{i-1, d}(X_{15:i-1, j} + X_{i, j}) ] \\
    &\quad+ p_d\left[ X_{15:i-1, j} + X_{i, j} \right] \\
    p_{i, s}X_{i, j} &= X_{i-1, j} p_{i, f}(p_{i-1, r} + p_{i-1, f}) + X_{15:i-1, j}(p_{i, f}p_{i-1, d} + p_{i, d}) \\
    &\quad+ p_{i, f}I(j=i-1) + p_{i, f}p_{i-1, f}I(j=i-2)
\end{align*}

In [2]:
def count_analysis(starcatch=True, event_15=False, dest_prevention=False):
    prob_chart = generate_prob_chart(starcatch, event_15, dest_prevention)
    count_chart = np.zeros((13, 13))  # 12->13 ~ 24->25
    np.fill_diagonal(count_chart, 1/prob_chart[12:, 0])
    count_chart = pd.DataFrame(
        count_chart,
        index=range(12, 25),
        columns=range(12, 25),
    )

    for start in range(15, 25):
        count_chart.loc[start, :start-1] = \
            count_chart.loc[start-1, :start-1] * prob_chart[start, 2] * prob_chart[start-1, [1, 2]].sum() + \
            count_chart.loc[:start-1, :start-1].sum(axis=0) * \
                (prob_chart[start, 2]*prob_chart[start-1, -1]+prob_chart[start, -1])
        
        count_chart.loc[start, start-2] += prob_chart[start, 2] * prob_chart[start-1, 2]
        count_chart.loc[start, start-1] += prob_chart[start, 2]
        count_chart.loc[start, :start-1] /= prob_chart[start, 0]

    return count_chart


Let $X_{i, j}$ be the destruction count at $j$ while enhancing StarForce $i \rightarrow i+1$.
\begin{align*}
    X_{i, j} &= p_{i, r}X_{i, j} + p_{i, f} \\
    &\quad\cdot\left[ p_{i-1, s}X_{i, j} + p_{i-1, r}(X_{i-1, j}+X_{i, j}) + p_{i-1, f}(X_{i-1, j}+X_{i, j}) + p_{i-1, d}(I(j=i-1) + X_{15:i-1, j} + X_{i, j}) \right] \\
    &\quad+ p_d\left[ I(j=i) + X_{15:i-1, j} + X_{i, j} \right] \\
    p_{i, s}X_{i, j} &= X_{i-1, j} p_{i, f}(p_{i-1, r} + p_{i-1, f}) + X_{15:i-1, j}(p_{i, f}p_{i-1, d} + p_{i, d}) + p_{i, d}I(j=i) + p_{i, f}p_{i-1, d}I(j=i-1)
\end{align*}

In [3]:
def dest_analysis(starcatch=True, event_15=False, dest_prevention=False):
    prob_chart = generate_prob_chart(starcatch, event_15, dest_prevention)
    dest_chart = np.zeros((10, 10))  # 15->16 ~ 24->25
    np.fill_diagonal(dest_chart, prob_chart[15:, -1]/prob_chart[15:, 0])
    dest_chart = pd.DataFrame(
        dest_chart,
        index=range(15, 25),
        columns=range(15, 25),
    )

    for start in range(16, 25):
        dest_chart.loc[start, :start-1] = \
            dest_chart.loc[start-1, :start-1] * prob_chart[start, 2] * prob_chart[start-1, [1, 2]].sum() + \
            dest_chart.loc[:start-1, :start-1].sum(axis=0) * \
                (prob_chart[start, 2]*prob_chart[start-1, -1]+prob_chart[start, -1])
        
        dest_chart.loc[start, start-1] += prob_chart[start, 2]*prob_chart[start-1, -1]
        dest_chart.loc[start, :start-1] /= prob_chart[start, 0]

    return dest_chart

In [4]:
def overall_analysis(
    start, end, analysis_1=15, analysis_2=16,
    starcatch=True, event_15=False, dest_prevention=False
):
    count_chart = count_analysis(starcatch, event_15, dest_prevention).loc[start:end-1, :].sum(axis=0)
    dest_chart = dest_analysis(starcatch, event_15, dest_prevention).loc[start:end-1, :].sum(axis=0)

    total_count = count_chart.sum()
    count_range = count_chart.loc[analysis_1:analysis_2].sum()
    count_range_prop = count_range / total_count

    total_dest = dest_chart.sum()
    dest_range = dest_chart.loc[analysis_1:analysis_2].sum()
    dest_range_prop = dest_range / total_dest

    bool_to_text = lambda x: "O" if x is True else "X"
    print(f'스타캐치:      {bool_to_text(starcatch)}')
    print(f'15+1 이벤트:   {bool_to_text(event_15)}')
    print(f'파괴 방지:     {bool_to_text(dest_prevention)}')
    print(f'{start:2d}성에서 {end:2d}성까지의 스타포스에 대한 분석은 다음과 같습니다.')
    print(f'총 시행 횟수({total_count:6.2f}회)의 {count_range_prop:5.1%}({count_range:6.2f}회)가 '\
          f'{analysis_1} ~ {analysis_2}에서 발생했습니다.')
    print(f'총 파괴 횟수({total_dest:6.2f}회)의 {dest_range_prop:5.1%}({dest_range:6.2f}회)가 '\
          f'{analysis_1} ~ {analysis_2}에서 발생했습니다.')

    return None


In [5]:
overall_analysis(15, 21, 15, 16, starcatch=False, event_15=False, dest_prevention=False)

스타캐치:      X
15+1 이벤트:   X
파괴 방지:     X
15성에서 21성까지의 스타포스에 대한 분석은 다음과 같습니다.
총 시행 횟수(284.08회)의 66.0%(187.52회)가 15 ~ 16에서 발생했습니다.
총 파괴 횟수(  4.98회)의 72.5%(  3.61회)가 15 ~ 16에서 발생했습니다.


In [6]:
overall_analysis(15, 21, 15, 16, starcatch=True, event_15=False, dest_prevention=False)

스타캐치:      O
15+1 이벤트:   X
파괴 방지:     X
15성에서 21성까지의 스타포스에 대한 분석은 다음과 같습니다.
총 시행 횟수(221.80회)의 64.6%(143.23회)가 15 ~ 16에서 발생했습니다.
총 파괴 횟수(  3.87회)의 69.4%(  2.68회)가 15 ~ 16에서 발생했습니다.


In [7]:
overall_analysis(15, 21, 15, 16, starcatch=True, event_15=True, dest_prevention=False)

스타캐치:      O
15+1 이벤트:   O
파괴 방지:     X
15성에서 21성까지의 스타포스에 대한 분석은 다음과 같습니다.
총 시행 횟수(150.72회)의 57.3%( 86.29회)가 15 ~ 16에서 발생했습니다.
총 파괴 횟수(  2.16회)의 45.1%(  0.97회)가 15 ~ 16에서 발생했습니다.


In [8]:
overall_analysis(15, 21, 15, 15, starcatch=True, event_15=True, dest_prevention=True)

스타캐치:      O
15+1 이벤트:   O
파괴 방지:     O
15성에서 21성까지의 스타포스에 대한 분석은 다음과 같습니다.
총 시행 횟수(142.66회)의 24.3%( 34.65회)가 15 ~ 15에서 발생했습니다.
총 파괴 횟수(  1.19회)의  0.0%(  0.00회)가 15 ~ 15에서 발생했습니다.


In [9]:
overall_analysis(15, 21, 16, 16, starcatch=True, event_15=True, dest_prevention=True)

스타캐치:      O
15+1 이벤트:   O
파괴 방지:     O
15성에서 21성까지의 스타포스에 대한 분석은 다음과 같습니다.
총 시행 횟수(142.66회)의 36.2%( 51.64회)가 16 ~ 16에서 발생했습니다.
총 파괴 횟수(  1.19회)의  0.0%(  0.00회)가 16 ~ 16에서 발생했습니다.


In [10]:
overall_analysis(15, 21, 15, 16, starcatch=True, event_15=True, dest_prevention=True)

스타캐치:      O
15+1 이벤트:   O
파괴 방지:     O
15성에서 21성까지의 스타포스에 대한 분석은 다음과 같습니다.
총 시행 횟수(142.66회)의 60.5%( 86.29회)가 15 ~ 16에서 발생했습니다.
총 파괴 횟수(  1.19회)의  0.0%(  0.00회)가 15 ~ 16에서 발생했습니다.
